# Experiment 2
models are mine
training iterations are theirs

In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

https://pytorch.org/tutorials/beginner/chatbot_tutorial.html?highlight=chatbot#define-evaluation

# Import

In [27]:
from pathlib import Path
import io, os

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torchtext
from torchtext.vocab import build_vocab_from_iterator, GloVe
from torch import optim
from nltk.translate import bleu

import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from typing import List

device = "cuda" if torch.cuda.is_available() else "cpu"

# Look at Data
Data already preprocessed collected from the original repo.

In [3]:
data_root = Path("data/processed")

In [4]:
with open(data_root / "src-train.txt") as f:
    src_train = [line.strip() for line in f]

with open(data_root / "tgt-train.txt") as f:
    tgt_train = [line.strip() for line in f]

with open(data_root / "src-dev.txt") as f:
    src_dev = [line.strip() for line in f]

with open(data_root / "tgt-dev.txt") as f:
    tgt_dev = [line.strip() for line in f]

with open(data_root / "src-test.txt") as f:
    src_test = [line.strip() for line in f]

with open(data_root / "tgt-test.txt") as f:
    tgt_test = [line.strip() for line in f]

print(len(src_train), len(src_dev), len(src_test))

70484 10570 11877


# Vocabulary

In [5]:
import datahandler as dh

In [6]:
vocab = dh.load_and_build_vocab(
    data_root / "src-train.txt", data_root / "tgt-train.txt"
)

In [7]:
pad_index = vocab["<PAD>"]
sos_index = vocab["<SOS>"]
eos_index = vocab["<EOS>"]

In [8]:
# a tensor of shape (vocab_size, embedding_dim)
embedding_vector = dh.load_pretrained_glove(vocab, cache="data/")

# Batch Dataloader

In [9]:
config = {"src_max_seq": 40, "tgt_max_seq": 15, "batch_size": 64, "lr": 1.0}

In [10]:
from tokenization import Tokenizer

In [11]:
tokenizer = Tokenizer(vocab, pad_index, sos_index, eos_index)

src_train_tensor, src_train_mask = tokenizer.encode(
    src_train, max_seq=config["src_max_seq"]
)

tgt_train_tensor, tgt_train_mask = tokenizer.encode(
    tgt_train, add_sos=True, add_eos=True, max_seq=config["tgt_max_seq"]
)

src_test_tensor, src_test_mask = tokenizer.encode(
    src_test, max_seq=config["src_max_seq"]
)
tgt_test_tensor, tgt_test_mask = tokenizer.encode(
    tgt_test, add_sos=True, add_eos=True, max_seq=config["tgt_max_seq"]
)

src_dev_tensor, src_dev_mask = tokenizer.encode(src_dev, max_seq=config["src_max_seq"])
tgt_dev_tensor, tgt_dev_mask = tokenizer.encode(
    tgt_dev, add_sos=True, add_eos=True, max_seq=config["tgt_max_seq"]
)

In [12]:
tokenizer.decode(src_train_tensor[0].unsqueeze(dim=0), keep_specials=False)

['a pub / <UNK> / , or public house is , despite its name , a private house , but is called a public house because it is licensed to sell alcohol to the general public .']

In [13]:
src_train[0]

'a pub / pʌb / , or public house is , despite its name , a private house , but is called a public house because it is licensed to sell alcohol to the general public .'

In [14]:
src_train_mask.shape

torch.Size([70484, 40])

In [15]:
class SentenceQuestionDataset(Dataset):
    def __init__(
        self,
        sentences: torch.Tensor,
        questions: torch.Tensor,
        sentences_mask=None,
        questions_mask=None,
    ):
        """
        Represents a dataset of text pairs for training or evaluating models that
        deal with relationships between text passages.

        Args:
            vocab (torchtext.vocab.Vocab): A pre-built vocabulary object
                containing word mappings from text to numerical representation.
            sentences (List[str]): A list of text passages (sentences, paragraphs, etc.).
            questions (List[str]): A list of corresponding questions related to the sentences.
            Ls (int, optional): The maximum length to which sentences will be
                truncated or padded during preprocessing (default: 150).
            Lq (int, optional): The maximum length to which questions will be
                truncated or padded during preprocessing (default: 50).
        """
        self.sentences = sentences
        self.questions = questions
        self.sentences_mask = sentences_mask
        self.questions_mask = questions_mask

    def __len__(self):
        return self.sentences.size(0)

    def __getitem__(self, index):
        return (
            self.sentences[index],
            self.questions[index],
            self.sentences_mask[index],
            self.questions_mask[index],
        )

In [16]:
train_ds = SentenceQuestionDataset(
    src_train_tensor, tgt_train_tensor, src_train_mask, tgt_train_mask
)
test_ds = SentenceQuestionDataset(
    src_test_tensor, tgt_train_tensor, src_test_mask, tgt_test_mask
)
dev_ds = SentenceQuestionDataset(
    src_dev_tensor, tgt_dev_tensor, src_dev_mask, tgt_dev_mask
)

In [17]:
train_dl = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=True)
print(len(train_dl))
dev_dl = DataLoader(dev_ds, batch_size=8, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

1102


# Model Def

## Attention

In [18]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_size, decoder_hidden_size):
        super().__init__()

        self.projection_layer = nn.Linear(encoder_hidden_size, decoder_hidden_size)

    def forward(self, encoder_output, decoder_output):
        """
        Args:
            encoder_output (torch.Tensor): (N, L, encoder_hidden_size)
            decoder_output (torch.Tensor): (N, 1, decoder_hidden_size)


        Returns:
            score (torch.Tensor): (N, L, 1)
        """
        # => (N, L, decoder_hidden_size)
        projection = self.projection_layer(encoder_output)
        # => (N, L, 1)
        score = projection @ decoder_output.transpose(1, 2)
        score = F.softmax(score, dim=1)

        return score

## Encoder

In [19]:
class Encoder(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding,
        embedding_dim,
        hidden_dim=8,
        bidirectional=False,
        num_layers=1,
    ):
        super().__init__()
        self.embedding = embedding
        self.encoder = nn.Sequential(
            self.embedding,
            nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                batch_first=True,
                bidirectional=bidirectional,
                num_layers=num_layers,
            ),
        )

    def forward(self, src: torch.Tensor):
        """
        Args:
            src (torch.Tensor): (N, Ls) A batch of source sentences represented as tensors.
        """
        # encoder_representation (N, Ls, d), hT => cT => (#direction * #layer, N, d) : hidden states from the last timestep
        encoder_out, (last_hidden_state, last_cell_state) = self.encoder(src)
        return encoder_out, last_hidden_state

## Decoder

In [20]:
class Decoder(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding,
        embedding_dim,
        hidden_dim=8,
        encoder_bidirectional=False,
        num_layers=1,
    ):
        super().__init__()
        self.embedding = embedding
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=False,
            num_layers=num_layers,
        )

        # self.attn_layer = nn.Linear(
        #     in_features=hidden_dim * 2 if bidirectional else hidden_dim,
        #     out_features=hidden_dim * 2 if bidirectional else hidden_dim,
        # )

        self.attention = Attention(
            hidden_dim * 2 if encoder_bidirectional else hidden_dim, hidden_dim
        )

        self.decoder_linear = nn.Sequential(
            # 3*hidden_dim because decoder_out and source context will be concatenated
            # this layer is Eq 5 in the Luong et. al. paper
            nn.Linear(
                hidden_dim * 3 if encoder_bidirectional else hidden_dim, hidden_dim
            ),
            nn.Tanh(),
            nn.Linear(hidden_dim, vocab_size),
        )

    def forward(self, encoder_out, target, last_hidden_state, last_cell_state):
        x = self.embedding(target)
        # => N, 1, d
        output, (ht, ct) = self.lstm(x, (last_hidden_state, last_cell_state))
        # => (N, Ls, 1)
        score = self.attention(encoder_out, output)
        # (N, Ls, 1) x (N, Ls, DH) => (N, Ls, DH) => (N, 1, DH)
        # Eq 4 from the Du et. al. paper (learning to ask)
        attn_based_ctx = (score * encoder_out).sum(dim=1).unsqueeze(dim=1)
        # => (N, 1, d ) & (N, 1, DH)
        concatenated = torch.cat((output, attn_based_ctx), dim=2).squeeze()
        # => (N, vocab_size)
        logit = self.decoder_linear(concatenated)

        return logit, ht

# Training Utils

## Single Step

In [33]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = torch.nn.functional.cross_entropy(inp, target, reduction="none")
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [36]:
def train(
    input_variable,
    lengths,
    target_variable,
    mask,
    max_target_len,
    encoder,
    decoder,
    embedding,
    encoder_optimizer,
    decoder_optimizer,
    batch_size,
    clip,
):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Lengths for RNN packing should always be on the CPU
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor(
        [[sos_index for _ in range(batch_size)]]
    ).transpose(0, 1)
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:num_layers]

    # Forward batch of sequences through decoder one time step at a time
    for t in range(max_target_len):
        decoder_output, decoder_hidden = decoder(
            encoder_outputs,
            decoder_input,
            decoder_hidden,
            torch.randn_like(decoder_hidden),
        )
        # Teacher forcing: next input is current target
        decoder_input = target_variable[:, t].view(-1, 1)
        # Calculate and accumulate loss
        mask_loss, nTotal = maskNLLLoss(
            decoder_output, target_variable[:, t], mask[:, t]
        )
        loss += mask_loss
        print_losses.append(mask_loss.item() * nTotal)
        n_totals += nTotal

    # Perform backpropagation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

## Training Loop

In [34]:
def trainIters(
    model_name,
    vocab,
    train_dl,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    embedding,
    encoder_n_layers,
    decoder_n_layers,
    save_dir,
    n_iteration,
    batch_size,
    print_every,
    save_every,
    clip,
    corpus_name,
    loadFilename,
):

    # Initializations
    print("Initializing ...")
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint["iteration"] + 1

    # Training loop
    print("Training...")
    for iteration, training_batch in enumerate(train_dl):
        # Extract fields from batch
        input_variable, target_variable, input_mask, target_mask = training_batch
        lengths = input_mask.sum(dim=1)

        # Run a training iteration with batch
        loss = train(
            input_variable,
            lengths,
            target_variable,
            target_mask,
            config["tgt_max_seq"],
            encoder,
            decoder,
            embedding,
            encoder_optimizer,
            decoder_optimizer,
            batch_size,
            clip,
        )
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print(
                "Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(
                    iteration, iteration / n_iteration * 100, print_loss_avg
                )
            )
            print_loss = 0

        # Save checkpoint
        if iteration % save_every == 0:
            directory = os.path.join(
                save_dir,
                model_name,
                corpus_name,
                "{}-{}_{}".format(encoder_n_layers, decoder_n_layers, hidden_size),
            )
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save(
                {
                    "iteration": iteration,
                    "en": encoder.state_dict(),
                    "de": decoder.state_dict(),
                    "en_opt": encoder_optimizer.state_dict(),
                    "de_opt": decoder_optimizer.state_dict(),
                    "loss": loss,
                    "voc_dict": vocab.get_stoi(),
                    "embedding": embedding.state_dict(),
                },
                os.path.join(directory, "{}_{}.tar".format(iteration, "checkpoint")),
            )

# Train Now

## Model Instantiate

In [29]:
vocab_size = len(vocab)
embedding_dim = 300
bidirectional = True
num_layers = 2
hidden_dim = 500
hidden_size = hidden_dim
batch_size = 64

In [43]:
embedding = nn.Embedding(vocab_size, embedding_dim)
encoder = Encoder(
    vocab_size,
    embedding,
    embedding_dim,
    hidden_dim,
    bidirectional,
    num_layers,
)
decoder = Decoder(
    vocab_size,
    embedding,
    embedding_dim,
    hidden_dim,
    bidirectional,
    num_layers,
)

## Run

In [44]:
# Configure training/optimization
model_name = 'cb_model'
corpus_name='squad'

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
save_dir = './save'

clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

# If you have CUDA, configure CUDA to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, vocab, train_dl, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, num_layers, num_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 0; Percent complete: 0.0%; Average loss: 10.8124


KeyboardInterrupt: 